# model design
- 2 conv layer with max pooling, activation=Relu

> ***Why do we use keras.layers.Flatten()?***   
컨볼루션 과정을 거치고 마지막에 fully connected에 입력으로 들어갈 때
1차원 데이터가 더 적합하다. 왜? FC도 1차원 노드들의 층으로 결합되어 있으니깐! 이라고 생각함.

> *Why do we use keras.layers.Dense()?*   
이건 찾아봐야겠다. 

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

n_channel_1=64    #num of filters of channel 1
n_channel_2=32    #num of filters of channel 2

n_dense=128
n_train_epoch=10

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))


model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

# model training

In [ ]:
# 모델 훈련
model.fit(x_train_norm, y_train, epochs=n_train_epoch)


# load test dataset

학습 데이터 불러올때 처럼 똑같이 해준다.
**학습데이터와 테스트데이터의 경로명이 다름에 주의!**

In [ ]:
def load_test_data(img_path):
    
    number_of_data=390+381+388   .
    img_size=28
    color=3
    
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/s/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=0   
        idx=idx+1

    for file in glob.iglob(img_path+'/r/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=1   
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/p/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=2   
        idx=idx+1
        
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/rsp"
(x_test, y_test)=load_test_data(image_dir_path)
x_test_norm = x_test/255.0


# Evaluation with Test dataset

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test_norm,y_test, verbose=2)

# ACC : 33.48% ...

> what was the problem?

**1. 데이터셋의 부족**
***
학습 데이터셋이 약 1000개 밖에 안되었다. 좀 더 많이 다양하게 찍어볼걸.




**2. 저해상도 데이터**
***
28X28 이미지를 사용하였는데.. 너무 해상도가 낮아 특징을 잡기 어려웠던게 아닐까?


conv 및 pooling 레이어를 더 추가도 해보고 필터 수도 조정하고 해보았다.
6개 레이어(3개 conv, 3개 pooling)이상으로 하기 위해선 이미지의 사이즈가 더 커야했다.
그렇기 때문에 레이어는 4개로 고정하고 필터 수 및 다른 하이퍼파라미터를 조정해봤으나
성능이 드라마틱하게 좋아지지는 않았다.

train데이터와 test데이터를 각각 왼손, 오른손으로 찍었는데 이러한 차이가 성능을 저하시킨 원인이 되었을 수도 있을 것 같다.

아직 CNN에 대한 지식이 많이 없어 어떤걸 손봐야 성능이 나아지는지 잘 모르겠다. :(
